In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
from preprocessing.Preprocessor import Preprocessor
from metrics.metrics import Metrics
pp = Preprocessor(spark)

In [3]:
spark

In [4]:
# Create ML Model

In [5]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# BoW

In [6]:
train = pp.to_bow('Cleaned Data/train.csv', min_frequency=275.)  # eg.: load in bag-of-words encoding
test = pp.to_bow('Cleaned Data/val.csv')

#train = pp.to_bow('/Users/jackrh2019/train.csv', min_frequency=500.) 
#test = pp.to_bow('/Users/jackrh2019/test.csv')

In [7]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1948,[0,1,2,3,4,...|    2|
|(1948,[0,1,6,7,16...|    0|
|(1948,[0,1,2,3,4,...|    0|
|(1948,[5,27,34,74...|    2|
|(1948,[0,1,2,4,5,...|    2|
+--------------------+-----+
only showing top 5 rows



In [8]:
train.cache()
test.cache()

DataFrame[features: vector, label: int]

In [9]:
# Chain indexers and tree in a Pipeline

In [10]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train)
featureIndexer=VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [11]:
model = pipeline.fit(train)

In [12]:
# Make predictions.
predictions = model.transform(test)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         1.0|(1948,[0,1,2,4,6,...|
|       0.0|         0.0|(1948,[0,1,2,3,4,...|
|       0.0|         1.0|(1948,[2,3,4,5,7,...|
|       0.0|         0.0|(1948,[0,1,2,3,4,...|
|       0.0|         0.0|(1948,[1,2,3,4,5,...|
+----------+------------+--------------------+
only showing top 5 rows



In [13]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Py4JJavaError: An error occurred while calling o410.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 78) (sachins-mbp.home executor driver): org.apache.spark.SparkException: Failed to execute user defined function(VectorIndexerModel$$Lambda$3654/0x0000000801dd3040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_subExpr_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scala.collec

In [ ]:
val_m = Metrics()
val_pred_and_labels = model.transform(test).select('prediction', 'label')
val_m.f1(val_pred_and_labels)

# IDF

In [ ]:
train_idf = pp.to_tfidf('Cleaned Data/train.csv', min_frequency=275.)
val_idf = pp.to_tfidf('Cleaned Data/val.csv')


In [ ]:
train_idf.show(5)

In [ ]:
train_idf.cache()
val_idf.cache()

In [ ]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_idf)
featureIndexer=VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_idf)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [ ]:
model = pipeline.fit(train_idf)

In [ ]:
val_m = Metrics()
val_pred_and_labels = model.transform(val_idf).select('prediction', 'label')
val_m.f1(val_pred_and_labels)

# word2vec

In [ ]:
train_2vec = pp.to_word2vec('Cleaned Data/train.csv', vector_size=256)
val_2vec = pp.to_word2vec(('Cleaned Data/train.csv'))


In [ ]:
train_2vec.show(5)

In [ ]:
train_2vec.cache()
val_2vec.cache()

In [ ]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_2vec)
featureIndexer=VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_2vec)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [ ]:
model = pipeline.fit(train_2vec)

In [ ]:
val_m = Metrics()
val_pred_and_labels = model.transform(val_2vec).select('prediction', 'label')
val_m.f1(val_pred_and_labels)